## RL agent Q-learning for TicTacToe env

The [Tic-Tac-Toe](https://github.com/MauroLuzzatto/OpenAI-Gym-TicTacToe-Environment) is a simple game environment that allows to train reinforcement learning agents. These notebook contains an implemetation of Q-learning with epsilon-greedy strategy for TicTacToe env.

In [166]:
# load the python modules
import time
import sys
import warnings

import gym
import numpy as np
from tqdm import tqdm
import gym_TicTacToe

from src.qagent import QLearningAgent
from src.play_tictactoe import play_tictactoe, play_tictactoe_with_random

from src.utils import (
    create_state_dictionary,
    reshape_state,
    save_qtable,
    load_qtable
)

# ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [167]:
class Player:
    def __init__(self, color, episodes: int):
        self.color = color
        self.reward_array = np.zeros(episodes)
        self.reset_reward()
        self.name = f"Player {color}"

    def reset_reward(self):
        self.reward = 0

In [168]:

# initialize the tictactoe environment
env = gym.envs.make("TTT-v0", small=-1, large=10)

In [169]:
state_dict = create_state_dictionary()
state_size = len(state_dict.keys())
action_size = env.action_space.n

Number of legal states: 12092


In [170]:
# set training parameters
episodes = 960_000
max_steps = 9

In [171]:
exploration_parameters = {
    "max_epsilon": 1.0,
    "min_epsilon": 0.0,
    "decay_rate": 0.00001,
}

In [172]:
qagent = QLearningAgent(exploration_parameters, state_size, action_size, learning_rate=0.1, gamma=0.99)

In [173]:
def check_for_potential_lose(state, color: int) -> bool:
        """check if after agent's move there is a lose position

        Args:
            color (int): of the player's enemy

        Returns:
            bool: indicating if this was a crucial move
        """
        state_check = np.copy(state)
        lose = False
        col = np.array([1,2])
        #enemy color
        enemy_color = color
        player_color = col[col != enemy_color][0]
        state_check[state_check == player_color] = -1
        state_check[state_check == enemy_color] = 1
        state_check = state_check.reshape(3,3)
        for ii in range(3):
            if (
                # check columns
                np.sum(state_check[:, ii]) == 2
                # check rows
                or np.sum(state_check[ii, :]) == 2
                # check diagonal
                or np.sum([state_check[0, 0], state_check[1, 1], state_check[2, 2]])
                == 2
                or np.sum([state_check[0, 2], state_check[1, 1], state_check[2, 0]])
                == 2
            ):
                lose = True
                break
        return lose

In [174]:
state = np.array([0,1,2,
                  0,1,2,
                  2,0,1])

check_for_potential_lose(state, color=1)

True

In [175]:
def play(qagent:QLearningAgent, player_color, state: int, action_space: np.array, last_turn: bool) -> tuple:
    action = qagent.get_action(state, action_space)

    # remove action from the action space
    action_space = action_space[action_space != action]

    new_state, reward, done, _ = env.step((action, player_color))

    col = np.array([1,2])
    # if done:
    #     print(new)
    #     reward -= 5
    if (done == False):
        if check_for_potential_lose(new_state, col[col != player_color][0]):
            reward += 4

    # if (last_turn == True) and (done == False):
    #     reward += 9     # Reward for draw

    # elif (last_turn == True) and (done == True):
    #     reward += 10
        # print(f"New_state:{new_state}, Reward:{reward}, Done:{done}")
    #TODO: maybe should change a marker after this agent turn 
    new_state = np.append(new_state, player_color)
    new_state = state_dict[reshape_state(new_state)] 

    qagent.qtable[state, action] = qagent.update_qtable(
        state, new_state, action, reward, done
    )
    # new state
    state = new_state
    return state, action_space, done

In [176]:
def play_random(qagent:QLearningAgent, player_color, state: int, action_space: np.array) -> tuple:
    action = np.random.choice(action_space)
    action_space = action_space[action_space != action]
    new_state, reward, done, _ = env.step((action, player_color))
    new_state = np.append(new_state, player_color)
    new_state = state_dict[reshape_state(new_state)]
    state = new_state
    return state, action_space, done

In [177]:
visited_states = np.zeros((state_size, 1))

In [178]:
lear_rate = 0.8
gamma = 0.9
qagent = QLearningAgent(exploration_parameters, state_size, action_size, learning_rate=lear_rate, gamma=gamma)

In [179]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
import random

start_time = time.time()

player_1 = Player(color=1, episodes=episodes)
player_2 = Player(color=2, episodes=episodes)

track_progress = np.zeros(episodes)

win_history = []

rewards = []
# lr = 0.4, gamma = 0.9, winrate = 0.64
# Learning rate: 0.5, Win rate: 0.72, Gamma: 0.9
# Learning rate: 0.6, Win rate: 0.5, Gamma: 0.9
# Learning rate: 0.7, Win rate: 0.66, Gamma: 0.9
# Learning rate: 0.65, Win rate: 0.76, Gamma: 0.8

# best 
# Learning rate: 0.8, Win rate: 0.8, Gamma: 0.9

# qagent_old = qagent
for episode in tqdm(range(episodes)):
    last_turn = False
    action_space = np.arange(9)
    player_1.reset_reward()
    player_2.reset_reward()

    # randomly change the order players
    start = np.random.choice([1,2])

    state, _ = env.reset()
    state = np.append(state, start)
    state = state_dict[reshape_state(state)]
    # if episode % 10_000 == 0:
    #     save_qtable(qagent.qtable, 'tables', "q_table_old")

    for _step in range(start, max_steps + start):
        if _step == max_steps + start - 1:
            last_turn = True
        # change a turn
        if _step % 2 == 0:
            # state, action_space, done = play_random(qagent, player_1.color, state, action_space)
            #qagent_old.qtable = load_qtable('tables', "q_table_old")
            state, action_space, done = play(qagent, player_1.color, state, action_space, last_turn)
        else:
            state, action_space, done = play(qagent, player_2.color, state, action_space, last_turn)
        visited_states[state] += 1
        if done == True:
            break

    # reduce epsilon for exporation-exploitation tradeoff
    qagent.update_epsilon(episode)

    #cur_win_rate, reward = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=100)

    #check how good is agent
    if episode % 25_000 == 0:
        num_games = 50
        cur_win_rate, reward = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=num_games)
        win_history.append(sum(cur_win_rate)/num_games)
        print("WinRate:", sum(cur_win_rate)/num_games)
        # rewards.append(reward)
        # clear_output(True)
        # # plt.title('eps = {:e}, mean reward = {:.1f}'.format(agent.epsilon, np.mean(rewards[-10:])))
        # plt.plot(rewards)
        # plt.show()
    if episode % 25_000 == 0:

        sum_q_table = np.sum(qagent.qtable)
        time_passed = round((time.time() - start_time) / 60.0, 2)

        print(
            f"episode: {episode}, \
            epsilon: {round(qagent.epsilon, 2)}, \
            sum q-table: {sum_q_table}, \
            elapsed time [min]: {time_passed},  \
            done [%]: {episode / episodes * 100} \
            "
        )


  0%|          | 0/960000 [00:00<?, ?it/s]

  0%|          | 43/960000 [00:00<1:05:41, 243.52it/s]

WinRate: 0.48
episode: 0,             epsilon: 1.0,             sum q-table: 53.27625481177885,             elapsed time [min]: 0.0,              done [%]: 0.0             


  3%|▎         | 25051/960000 [01:07<50:31, 308.39it/s]

WinRate: 0.64
episode: 25000,             epsilon: 0.78,             sum q-table: 282065.97713629797,             elapsed time [min]: 1.13,              done [%]: 2.604166666666667             


...

WinRate: 0.96
episode: 575000,             epsilon: 0.0,             sum q-table: 355563.845132893,             elapsed time [min]: 30.8,              done [%]: 59.895833333333336             


 63%|██████▎   | 600070/960000 [32:11<20:32, 292.09it/s]

WinRate: 0.72
episode: 600000,             epsilon: 0.0,             sum q-table: 355563.89429175056,             elapsed time [min]: 32.18,              done [%]: 62.5             


 64%|██████▍   | 618841/960000 [33:13<18:19, 310.36it/s]


KeyboardInterrupt: 

In [180]:
visited_states.shape[0]
print("Percent:",100*np.sum(visited_states > 0)/visited_states.shape[0])

Percent: 90.58881905391995


In [183]:
num_games = 1000
cur_win_rate, _ = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=num_games)
win_history.append(sum(cur_win_rate)/num_games)
print("WinRate:", sum(cur_win_rate)/num_games)

WinRate: 0.84


In [314]:
qtable = qagent.qtable
save_qtable(qtable, 'tables', "q_table_best2")

q_table_best2.npy saved!


In [10]:
qtable = load_qtable('tables', "q_table_best2")

In [191]:
#check how correct is q-table

state = np.random.choice(np.arange(env.observation_space.n))
# state_dict[state]
print(state)

key = list(filter(lambda x: state_dict[x] == state, state_dict))[0]
print(np.array(key[:-1]).reshape(3,3))
print("Turn was:", key[-1])
print(np.round(qtable[state].reshape(3,3),1))

state_pure = np.array(key[:-1])
action_space = np.where(state_pure == 0)[0]
array = np.array(qtable[state, :])
order = array.argsort()
ranks = order.argsort()
max_value_rank = np.min(ranks[action_space])
action = np.where(ranks == max_value_rank)[0][0]
action

4066
[[1 0 0]
 [0 0 2]
 [1 0 0]]
Turn was: 1
[[ 0.  16.7 19.9]
 [16.7 21.3  0. ]
 [ 0.  17.1 21.2]]
[ 0.   16.66 19.87 16.68 21.32  0.    0.   17.07 21.15]
4


1

In [165]:
play_tictactoe(env, qtable, state_dict, num_test_games=1)

Agent beginns
--------------------
╒═══╤═══╤═══╕
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
--------------------
move Agent
Action: 1
╒═══╤═══╤═══╕
│ - │ O │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛


--------------------
Move Human
Action: 0
╒═══╤═══╤═══╕
│ X │ O │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
-1


--------------------
move Agent
Action: 6
╒═══╤═══╤═══╕
│ X │ O │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ O │ - │ - │
╘═══╧═══╧═══╛


--------------------
Move Human
Action: 5
╒═══╤═══╤═══╕
│ X │ O │ - │
├───┼───┼───┤
│ - │ - │ X │
├───┼───┼───┤
│ O │ - │ - │
╘═══╧═══╧═══╛
-1


--------------------
move Agent
Action: 3
╒═══╤═══╤═══╕
│ X │ O │ - │
├───┼───┼───┤
│ O │ - │ X │
├───┼───┼───┤
│ O │ - │ - │
╘═══╧═══╧═══╛


--------------------
Move Human
Action: 4
╒═══╤═══╤═══╕
│ X │ O │ - │
├───┼───┼───┤
│ O │ X │ X │
├───┼───┼───┤
│ O │ - │ - │
╘═══╧═══╧═══╛
-1


------